## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import map_key

# Configure gmaps API key
gmaps.configure(api_key=map_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Klaksvík,FO,4/12/2022 3:20,62.2266,-6.5890,5.51,79,100,11.70,overcast clouds
1,1,Rikitea,PF,4/12/2022 3:17,-23.1203,-134.9692,26.15,71,1,7.06,clear sky
2,2,Fortuna,US,4/12/2022 3:20,40.5982,-124.1573,6.62,83,100,6.69,overcast clouds
3,3,Kobryn,BY,4/12/2022 3:20,52.2138,24.3564,-0.31,85,24,2.67,few clouds
4,4,Port Alfred,ZA,4/12/2022 3:20,-33.5906,26.8910,18.43,93,100,0.89,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = float(input("What is the prefered Maximum temperature?"))
min_temp = float(input("What is the prefered Miniimum temperature?"))

What is the prefered Maximum temperature?20
What is the prefered Miniimum temperature?15


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
prefered_cities_df = city_data_df.loc[((city_data_df["Max Temp"] >=min_temp) & (city_data_df["Max Temp"] <=max_temp))]
prefered_cities_df

,City_ID,City,Country,Date,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Port Alfred,ZA,4/12/2022 3:20,-33.5906,26.8910,18.43,93,100,0.89,overcast clouds
12,12,Hermanus,ZA,4/12/2022 3:20,-34.4187,19.2345,18.91,63,85,2.25,overcast clouds
22,22,Ayní,TJ,4/12/2022 3:20,39.3941,68.5377,15.20,63,9,2.88,clear sky
25,25,San Vicente,AR,4/12/2022 3:20,-26.6167,-54.1333,17.39,100,100,1.10,moderate rain
27,27,Castro,BR,4/12/2022 3:20,-24.7911,-50.0119,16.68,98,72,1.52,broken clouds
...,...,...,...,...,...,...,...,...,...,...,...
635,635,Pisco,PE,4/12/2022 3:24,-13.7000,-76.2167,18.03,82,0,6.17,clear sky
641,641,Asilah,MA,4/12/2022 3:24,35.4650,-6.0348,15.98,86,26,4.16,scattered clouds
648,648,Chicama,PE,4/12/2022 3:24,-7.8447,-79.1469,18.30,81,91,3.28,overcast clouds
649,649,Oranjemund,NaN,4/12/2022 3:24,-28.5500,16.4333,18.85,55,0,1.35,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
prefered_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Date                   0
Latitude               0
Longitude              0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = prefered_cities_df.dropna()


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Latitude", "Longitude"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Latitude,Longitude,Hotel Name
4,Port Alfred,ZA,18.43,overcast clouds,-33.5906,26.8910,
12,Hermanus,ZA,18.91,overcast clouds,-34.4187,19.2345,
22,Ayní,TJ,15.20,clear sky,39.3941,68.5377,
25,San Vicente,AR,17.39,moderate rain,-26.6167,-54.1333,
27,Castro,BR,16.68,broken clouds,-24.7911,-50.0119,
29,Nouadhibou,MR,17.41,overcast clouds,20.9310,-17.0347,
33,New Norfolk,AU,15.77,moderate rain,-42.7826,147.0587,
34,San Juan,AR,15.34,clear sky,-31.5375,-68.5364,
35,Cape Town,ZA,17.20,clear sky,-33.9258,18.4232,
40,Ribeira Grande,PT,15.90,overcast clouds,38.5167,-28.7000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": map_key,
    'location':""
}

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    coordinates = str(row['Latitude'])+","+str(row['Longitude'])
    
    params['location']=coordinates
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    baseURL = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    # 6e. Make request and retrieve the JSON data from the search. 
    lodge = requests.get(baseURL, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = lodge['results'][0]['name']
    except:
        hotel_df.loc[index, "Hotel Name"] = "Hotel Not Found"

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name'] !='Hotel Not Found']
clean_hotel_df

,City,Country,Max Temp,Current Description,Latitude,Longitude,Hotel Name
4,Port Alfred,ZA,18.43,overcast clouds,-33.5906,26.8910,The Halyards Hotel
12,Hermanus,ZA,18.91,overcast clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
22,Ayní,TJ,15.20,clear sky,39.3941,68.5377,Меҳмонхона СУҒД
25,San Vicente,AR,17.39,moderate rain,-26.6167,-54.1333,Complejo Turístico Farelluk
27,Castro,BR,16.68,broken clouds,-24.7911,-50.0119,CHACARA BAILLY
...,...,...,...,...,...,...,...
622,Srinagar,IN,17.71,overcast clouds,34.0833,74.8167,Hotel New Green View
635,Pisco,PE,18.03,clear sky,-13.7000,-76.2167,Hotel San Isidro Oficial
641,Asilah,MA,15.98,scattered clouds,35.4650,-6.0348,Hotel Zelis
648,Chicama,PE,18.30,overcast clouds,-7.8447,-79.1469,Balihai Bungalows


In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")


In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °C</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Latitude", "Longitude"]]

In [12]:
# 11a. Add a marker layer for each city to the map.
coordinates = clean_hotel_df[['Latitude','Longitude']]
marker_layer = gmaps.marker_layer(coordinates, info_box_content = hotel_info)
fig = gmaps.figure(center = (30.0,-31.0), zoom_level = 1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))